In [1]:
import os
from fastcore.xtras import Path
from dotenv import load_dotenv, find_dotenv
import PyPDF2
import fitz
from pypdf import PdfReader, PdfWriter
from IPython.display import display_pdf, HTML, IFrame


In [2]:
def compress_images(pdf):    
    reader = PdfReader(pdf)
    writer = PdfWriter()
    
    for page in reader.pages:
        writer.add_page(page)
    
    for page in writer.pages:
        for img in page.images:
            img.replace(img.image, quality=80)
    
    if reader.metadata is not None:
        writer.add_metadata(reader.metadata)
    
    with open(pdf, "wb") as fp:
        writer.write(fp)

In [3]:
def remove_last_page(pdf):
    pdf_writer = PyPDF2.PdfWriter()

    pdf_reader = PyPDF2.PdfReader(pdf)
    num_pages = len(pdf_reader.pages)

    if num_pages > 1:
        num_pages -= 1

    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        pdf_writer.add_page(page)

    with open(pdf, "wb") as out_file:
        pdf_writer.write(out_file)


In [4]:
def optimize_pdf(input_path, output_path, remove_last_page=False):
    doc = fitz.open(input_path)
    if remove_last_page and len(doc) > 1:
        doc.delete_page(len(doc) - 1)
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(alpha=False)
        img = fitz.Pixmap(pix, 0)  # Convert to grayscale
        page.insert_image(page.rect, pixmap=img, keep_proportion=True)

    doc.save(output_path, garbage=4, deflate=True, clean=True)

In [5]:
def merge_pdfs(pdf_list, output_path, remove_last_page=False):
    pdf_writer = PyPDF2.PdfWriter()

    for pdf in pdf_list:
        pdf_reader = PyPDF2.PdfReader(pdf)
        num_pages = len(pdf_reader.pages)

        if remove_last_page and num_pages > 1:
            num_pages -= 1

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            pdf_writer.add_page(page)

    with open(output_path, "wb") as out_file:
        pdf_writer.write(out_file)


In [6]:
def merge_pdfs(pdf_list, output_path, remove_last_page=False):
    pdf_writer = PyPDF2.PdfWriter()

    for pdf in pdf_list:
        pdf_reader = PyPDF2.PdfReader(pdf)
        num_pages = len(pdf_reader.pages)

        if remove_last_page and num_pages > 1:
            num_pages -= 1

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            pdf_writer.add_page(page)

    with open(output_path, "wb") as out_file:
        pdf_writer.write(out_file)


In [7]:
load_dotenv(find_dotenv(), override=True)
FOLDER = Path(os.environ["FOLDER"])
PDFS = FOLDER / 'screenshots'
PDFS.ls()


(#1750) [Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B00BL63176.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B08G1V1KZR.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B09QH71RZR.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B09QSB3LWX.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B09V6HMJBC.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B09V88XWFR.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B09ZF9XV3V.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B0B56T6WYQ.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B0BRQKGTVM.pdf'),Path('D:/OneDrive - ANATEL/Regulatron/Resultados/screenshots/amazon_20240708_B0BRQMKH5L.pdf')...]

In [8]:
MARKETPLACES = ['amazon', 'americanas', 'carrefour', 'magalu', 'ml', 'casasbahia']
DATE = '20240708'

In [10]:
for mk in MARKETPLACES:
    prefix = f'{mk}_{DATE}'
    pdf_files = PDFS.ls().filter(lambda file: file.suffix == ".pdf" and file.stem.startswith(prefix))
    # pdf_files.map(Path.unlink)
    # pdf_files.map(compress_images)
    merge_pdfs(
        pdf_files,
        FOLDER.parent / "issues" / "screenshots" / f"{mk}_{DATE}.pdf",
    )